In [1]:
import os
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
path = os.path.join(r'C:\Users\Я\Desktop\читы жизнь\Study\Programming\0_Jupyter\Project_Credit_Card\creditcard.csv')
df = pd.read_csv(path)
# Оставляем в датафрейме наиболее важные фичи
# df = df[['V17','V12','V14','V10','V11','V16','V18','V9','Class']]назы
#df = df.drop(['Amount','Time'],axis=1)

In [22]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [24]:
df.head().iloc[:,['Time','Amount']]

IndexError: .iloc requires numeric indexers, got ['Time' 'Amount']

* Разделим датасет на тестовую и тренировочную выборку

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1], test_size=0.22,
    random_state=42, stratify=df.iloc[:,-1])

df_train = pd.concat([X_train, y_train], axis=1)

# создадим отдельный датафрейм, в который запишем все объекты с классом 1
df_class_fraud = df_train[(df_train.Class == 1)]
# создадим отдельный датафрейм, в который запишем все объекты с классом 0
df_class_norm = df_train[df_train.Class == 0]
# создадим датафрейм в котором не будет параметра класс
X = df.drop(['Class'],axis=1)

* Появилась идея, что можно на время убрать из датасета строки с классом '1'.
* После этого можно будет провести обычный отбор фичей, и очистку датасета от выбросов
* Если первый пункт не выполнить, то тогда в датасете останется очень мало объектов с классом '1', или не останется вовсе. Так дизбаланс классов станет еще более явным

* Далее будем производить очистку 
* Использую 2 способа борьбы с аномалиями, для этого создам 2 версии датасета: для одного и другого способа.

In [4]:
def quantile(df_cut, X):
    for i in X.columns:
        a = df_cut[i].quantile(0.25)
        b = df_cut[i].quantile(0.75)
        df_cut = df_cut[(df_cut[i] > a - 1.5 * (b - a)) & (df_cut[i] < b + 1.5 * (b - a))]
    return df_cut
        

def mean_std(df_cut, X):
    for i in X.columns:
        left_side = df_cut[i].mean() - 5 * df_cut[i].std()
        right_side = df_cut[i].mean() + 5 * df_cut[i].std()

        df_cut = df_cut[(df_cut[i] > left_side) & (df_cut[i] < (right_side))]
    return df_cut
# df_cut.info(), df_cut['Class'].value_counts()

In [5]:
# создадим 2 версии датафрейма, каждую из них почистим от выбросов конкретным методом
df_class_norm_quantile = df_class_norm
df_class_norm_mean_std = df_class_norm

In [6]:
l = 3 
while l !=0:
    first_value = df_class_norm_quantile.shape[0]
    df_class_norm_quantile = quantile(df_class_norm_quantile, X)
    second_value = df_class_norm_quantile.shape[0]
    l = first_value - second_value
df_class_norm_quantile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35117 entries, 39153 to 29062
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      35117 non-null  float64
 1   V2      35117 non-null  float64
 2   V3      35117 non-null  float64
 3   V4      35117 non-null  float64
 4   V5      35117 non-null  float64
 5   V6      35117 non-null  float64
 6   V7      35117 non-null  float64
 7   V8      35117 non-null  float64
 8   V9      35117 non-null  float64
 9   V10     35117 non-null  float64
 10  V11     35117 non-null  float64
 11  V12     35117 non-null  float64
 12  V13     35117 non-null  float64
 13  V14     35117 non-null  float64
 14  V15     35117 non-null  float64
 15  V16     35117 non-null  float64
 16  V17     35117 non-null  float64
 17  V18     35117 non-null  float64
 18  V19     35117 non-null  float64
 19  V20     35117 non-null  float64
 20  V21     35117 non-null  float64
 21  V22     35117 non-null  float64

In [7]:
l = 3 
while l !=0:
    first_value = df_class_norm_mean_std.shape[0]
    df_class_norm_mean_std = mean_std(df_class_norm_mean_std, X)
    second_value = df_class_norm_mean_std.shape[0]
    l = first_value - second_value
df_class_norm_mean_std.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201898 entries, 39153 to 17677
Data columns (total 29 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      201898 non-null  float64
 1   V2      201898 non-null  float64
 2   V3      201898 non-null  float64
 3   V4      201898 non-null  float64
 4   V5      201898 non-null  float64
 5   V6      201898 non-null  float64
 6   V7      201898 non-null  float64
 7   V8      201898 non-null  float64
 8   V9      201898 non-null  float64
 9   V10     201898 non-null  float64
 10  V11     201898 non-null  float64
 11  V12     201898 non-null  float64
 12  V13     201898 non-null  float64
 13  V14     201898 non-null  float64
 14  V15     201898 non-null  float64
 15  V16     201898 non-null  float64
 16  V17     201898 non-null  float64
 17  V18     201898 non-null  float64
 18  V19     201898 non-null  float64
 19  V20     201898 non-null  float64
 20  V21     201898 non-null  float64
 21  V22    

* Соединим наши датасеты в тренировочный датасет

In [8]:
# # Сначала попробуем датасет почищеный методом квартилей
# df_train = pd.concat([df_class_fraud, df_class_norm_quantile], axis=0)
# from sklearn.utils import shuffle
# df_train = shuffle(df_train)
# X_train = df_train.iloc[:,:-1]
# y_train = df_train.iloc[:,-1]
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [9]:
# Теперь методом среднего и отклонения
df_train = pd.concat([df_class_fraud, df_class_norm_mean_std], axis=0)
from sklearn.utils import shuffle
df_train = shuffle(df_train)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((202282, 28), (202282,), (62658, 28), (62658,))

In [10]:
# X_train = X_train.loc[:,['V14','V17']]
# X_test = X_test.loc[:,['V14','V17']]
# y_train.value_counts()

In [11]:
# clf = DecisionTreeClassifier(max_depth=3)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

In [12]:
# clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,
#     max_depth=1, random_state=0)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

clf = RandomForestClassifier(max_depth=3, class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [13]:
import sklearn.externals
import joblib

In [14]:
# Сохранение модели
# filename = 'knn_model.sav'
# joblib.dump(clf, filename)



In [18]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
# define dataset
# define the model
model = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.995 (0.003)


In [32]:
# fit the model on the whole dataset
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [33]:
confusion_matrix(y_test, y_pred)

array([[62114,   436],
       [   44,    64]], dtype=int64)

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     62550
           1       0.13      0.59      0.21       108

    accuracy                           0.99     62658
   macro avg       0.56      0.79      0.60     62658
weighted avg       1.00      0.99      0.99     62658



In [ ]:
# clf = KNeighborsClassifier(n_neighbors=17, weights='distance',metric='manhattan')
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)